In [5]:
import math
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
import matplotlib.pylab as plt
import time
import os

In [2]:
def random_mini_batches(X, Y, mini_batch_size=64, seed=0):

    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0], m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = int(math.floor(m/mini_batch_size))
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches

def load_data(af_points='af_points',
              af_labels='af_labels',
              label_afs='label_afs'):

    with open(af_points + '_training.pickle', 'rb') as fid:
        af_data_dic = pickle.load(fid, encoding='latin1')
    with open(af_labels + '_training.pickle', 'rb') as fid:
        af_label = pickle.load(fid, encoding='latin1')
    with open(label_afs + '_training.pickle', 'rb') as fid:
        label_af = pickle.load(fid, encoding='latin1')

    with open(af_points + '_test.pickle', 'rb') as fid:
        af_data_dic_test = pickle.load(fid, encoding='latin1')
    with open(af_labels + '_test.pickle', 'rb') as fid:
        af_label_test = pickle.load(fid, encoding='latin1')
    with open(label_afs + '_test.pickle', 'rb') as fid:
        label_af_test = pickle.load(fid, encoding='latin1')

    return af_data_dic, af_label, label_af, af_data_dic_test, af_label_test, label_af_test

def hyper_parameters(lamda=0.0, learning_rate=0.00005,
                     epochs=10000, batch_size=1024,
                     beta_1=0.95, beta_2=0.995,
                     layers=[512, 450, 400, 350, 300, 250, 150, 100, 4]):
    hp = {'lamda': lamda,
          'lr': learning_rate,
          'epochs': epochs,
          'batch_size': batch_size,
          'adams_beta': [beta_1, beta_2],
          'layers': layers}

    return hp

def train_dev_test_sets(df_loc='raw_af_data', s_train=0.95,
                        alpha_range=[-5, 15], Re_range=[500000, 10000000],
                        inputs_list=['af', 're', 'a'], outputs_list=['cl', 'cd', 'cdp', 'cm'],
                        normalize=['re', 'a', 'cl', 'cd', 'cdp', 'cm']):
    sets = {}

    # create input
    data_train = pd.read_csv(df_loc+'_training.txt')
    index = data_train.index
    h = data_train.copy()
    # modify data
    h = h[(h['a'] > alpha_range[0]) & (h['a'] < alpha_range[1])]
    h = h[(h['re'] > Re_range[0]) & (h['re'] < Re_range[1])]
    # normalize the desired columns
    n_params = {}


    '''
    for col in normalize:
        mu = h[col].mean()
        sigma = h[col].std()
        h[col] = (h[col] - mu) / sigma
        n_params[col] = {'mu': mu, 'sigma': sigma}
    '''
    h['re'] = h['re'] / 8000000.
    h['a'] = h['a'] / 30.
    # shuffle the data three times
    # shuffle the data three times
    h = h.sample(frac=1, axis=0).reset_index(drop=True)
    h = h.reindex(np.random.permutation(h.index)).reset_index(drop=True)
    ## third shuffle
    inputs_train = h.sample(frac=s_train)
    remaining = h.drop(inputs_train.index)
    inputs_train = inputs_train.reset_index(drop=True)
    inputs_dev = remaining.sample(frac=(1 - s_train)).reset_index(drop=True)

    # create the train and dev sets
    sets['x_train'] = inputs_train[inputs_list].values.transpose()
    sets['y_train'] = inputs_train[outputs_list].values.transpose()
    sets['m_train'] = sets['x_train'].shape[1]

    sets['x_dev'] = inputs_dev[inputs_list].values.transpose()
    sets['y_dev'] = inputs_dev[outputs_list].values.transpose()
    sets['m_dev'] = sets['x_dev'].shape[1]

    # create the test set
    data_test = pd.read_csv(df_loc+'_test.txt')
    h = data_test.copy()
    # modify data
    h = h[(h['a'] > alpha_range[0]) & (h['a'] < alpha_range[1])]
    h = h[(h['re'] > Re_range[0]) & (h['re'] < Re_range[1])]
    '''
    for col in normalize:
        h[col] = (h[col] - n_params[col]['mu']) / n_params[col]['sigma']
    '''
    h['re'] = h['re'] / 8000000.
    h['a'] = h['a'] / 30.

    sets['x_test'] = h[inputs_list].values.transpose()
    sets['y_test'] = h[outputs_list].values.transpose()
    sets['m_test'] = sets['x_test'].shape[1]

    return n_params, sets

def create_network(sets, label_af, af_data_dic, layers=[512, 256, 200, 128, 64, 32, 4], lamda=0.8, seed=0):
    network = {}
    tf.reset_default_graph()
    network['n'] = af_data_dic[label_af[0]]['input'].shape[1] + 2
    network['A0'] = tf.placeholder('float64', shape=(network['n'], None))
    network['y'] = tf.placeholder('float64', shape=(sets['y_train'].shape[0], None))
    a = [network['n']]
    a.extend(layers)
    layers = a
    network['layers'] = layers
    for i in range(1, len(layers)):

        network['W' + str(i)] = tf.get_variable("W" + str(i), (layers[i], layers[i - 1]),
                                                initializer=tf.contrib.layers.xavier_initializer(seed=seed),
                                                dtype=tf.float64)
        network['b' + str(i)] = tf.get_variable("b" + str(i), [layers[i], 1],
                                                initializer=tf.zeros_initializer(), dtype=tf.float64)
        network['Z' + str(i)] = tf.add(tf.matmul(network['W' + str(i)], network['A' + str(i - 1)]),
                                       network['b' + str(i)])
        if i != len(layers) - 1:
            network['A' + str(i)] = tf.nn.leaky_relu(network['Z' + str(i)])
        else:
            network['A' + str(i)] = network['Z' + str(i)]
        if i == 1:
            network['reg'] = tf.nn.l2_loss(network['W' + str(i)])
        else:
            network['reg'] += tf.nn.l2_loss(network['W' + str(i)])

    # calculate cost
    network['cost'] = tf.reduce_sum((network['y'] - network['A' + str(len(layers) - 1)]) ** 2.)
    # compute reqularized loss
    network['cost'] = tf.reduce_mean(network['cost'] + lamda * network['reg'])

    return network


def test_test_set(network, sets, sess, af_data_dic, label_af, results='res.txt', label='test'):
    x_af = sets['x_test'][0, :].astype(int)
    x_af = np.array([af_data_dic[label_af[i]]['input'].flatten() for i in x_af]).transpose()
    x_temp = sets['x_test'][1:3, :]
    x_test_n = np.concatenate((x_af, x_temp), axis=0)

    predict = network['y'] - network['A' + str(len(network['layers']) - 1)]
    error = sess.run(predict, feed_dict={network['A0']: x_test_n, network['y']: sets['y_test']})
    fid = open(results, 'a')
    fid.write('{} infinity norm test cl : {}\n'.format(label, abs(error[0, :]).max()))
    fid.write('{} infinity norm test cd : {}\n'.format(label, abs(error[1, :]).max()))
    fid.write('{} infinity norm test cdp: {}\n'.format(label, abs(error[2, :]).max()))
    fid.write('{} infinity norm test cm : {}\n\n'.format(label, abs(error[3, :]).max()))
    fid.write('{} cl  std {} mean {}\n'.format(label, abs(error[0, :]).std(), abs(error[0, :]).mean()))
    fid.write('{} cd  std {} mean {}\n'.format(label, ((sets['y_test']-np.array(error[1, :]))/sets['y_test']).std(), ((sets['y_test']-np.array(error[1, :]))/sets['y_test']).mean()))
    fid.write('{} cdp std {} mean {}\n'.format(label, ((sets['y_test']-error[2, :])/sets['y_test']).std(), ((sets['y_test']-error[2, :])/sets['y_test']).mean()))
    fid.write('{} cm  std {} mean {}\n\n'.format(label, error[3, :].std(), error[3, :].mean()))
    fid.close()

    return


def test_dev_set(network, sets, sess, af_data_dic, label_af, results='res.txt', label='dev'):

    # create x_dev
    x_af = sets['x_dev'][0, :].astype(int)
    x_af = np.array([af_data_dic[label_af[i]]['input'].flatten() for i in x_af]).transpose()
    x_temp = sets['x_dev'][1:3, :]
    x_dex_n = np.concatenate((x_af, x_temp), axis=0)

    # predict the error
    predict = tf.abs(network['y'] - network['A' + str(len(network['layers']) - 1)])
    error = sess.run(predict, feed_dict={network['A0']: x_dex_n, network['y']: sets['y_dev']})
    # print(error.shape)
    # time.sleep(1)
    fid = open(results, 'a')
    fid.write('infinity norm:\n\n')
    fid.write('{} infinity norm cl : {}\n'.format(label, error[0, :].max()))
    fid.write('{} infinity norm cd : {}\n'.format(label, error[1, :].max()))
    fid.write('{} infinity norm cdp: {}\n'.format(label, error[2, :].max()))
    fid.write('{} infinity norm cm : {}\n'.format(label, error[3, :].max()))
    fid.write('{} cl  std {} mean {}\n'.format(label, error[0, :].std(), error[0, :].mean()))
    fid.write('{} cd  std {} mean {}\n'.format(label, error[1, :].std(), error[1, :].mean()))
    fid.write('{} cdp std {} mean {}\n'.format(label, error[2, :].std(), error[2, :].mean()))
    fid.write('{} cm  std {} mean {}\n\n'.format(label, error[3, :].std(), error[3, :].mean()))
    fid.close()

    return

In [3]:
af_points='af_points'
af_labels='af_labels'
label_afs='label_afs'
lamda=0.0
learning_rate=0.00005
epochs=100
batch_size=1024
beta_1=0.95
beta_2=0.995
layers=[250, 150, 10, 4]
df_loc='raw_af_data'
s_train = 0.95
alpha_range = [-5, 15]
Re_range = [500000, 10000000]
inputs_list = ['af', 're', 'a']
outputs_list = ['cl', 'cd', 'cdp', 'cm']
normalize = ['re', 'a', 'cl', 'cd', 'cdp', 'cm']

In [19]:
af_data_dic, af_label, label_af, af_data_dic_test, af_label_test, label_af_test = load_data(
              af_points=af_points,
              af_labels=af_labels,
              label_afs=label_afs)

hp = hyper_parameters(lamda=lamda, learning_rate=learning_rate,
                      epochs=epochs, batch_size=batch_size,
                      beta_1=beta_1, beta_2=beta_2, layers=layers)

n_params, sets = train_dev_test_sets(df_loc=df_loc, s_train=s_train, alpha_range=alpha_range,
                                     Re_range=Re_range, inputs_list=inputs_list, 
                                     outputs_list=outputs_list)

tf.reset_default_graph()
network = create_network(sets, label_af, af_data_dic, hp['layers'], hp['lamda'])

optimizer = tf.train.AdamOptimizer(learning_rate=hp['lr'],
                                   beta1=hp['adams_beta'][0],
                                   beta2=hp['adams_beta'][0]).minimize(network['cost'])

init = tf.global_variables_initializer()

saver = tf.train.Saver()

seed = 0
costs = []
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)

    for epoch in range(hp['epochs']):
        epoch_cost = 0.
        n_batch = int(sets['m_train'] / hp['batch_size'])
        seed = seed + 1
        minibatches = random_mini_batches(sets['x_train'], sets['y_train'], hp['batch_size'], seed)
        for minibatch in minibatches:
            (minibatch_x, minibatch_y) = minibatch
            # convert the x_s
            x_af = minibatch_x[0, :].astype(int)
            x_af = np.array([af_data_dic[label_af[i]]['input'].flatten() for i in x_af]).transpose()
            x_temp = minibatch_x[1:3, :]
            minibatch_x = np.concatenate((x_af, x_temp), axis=0)
            _, minibatch_cost = sess.run([optimizer, network['cost']], feed_dict={network['A0']: minibatch_x,
                                                                                  network['y']: minibatch_y})
            epoch_cost += minibatch_cost / n_batch
        
        print(sess.run(network['W' + str(len(network['layers']) - 1)]))
        
        # Print the cost every epoch
        if epoch % 1 == 0:
            cp = saver.save(sess, 'cp/cp_test.ckpt')
            fid = open('res.txt', 'a')
            fid.write("Cost after epoch {}: {}\n".format(epoch, epoch_cost))
            fid.close()
        if epoch % 1 == 0:
            costs.append(epoch_cost)
            print(epoch_cost)

        if epoch % 10 == 0:
            #cp = saver.save(sess, 'cp/cp_test.ckpt')
            test_dev_set(network, sets, sess, af_data_dic, label_af, results='res.txt', label='dev')

        if epoch % 20 == 0:
            test_test_set(network, sets, sess, af_data_dic_test, label_af_test, results='res.txt', label='test')


W0909 00:49:16.018850 140608746923840 deprecation.py:323] From /home/ariya/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[[ 0.41607852  0.40129075 -0.15619339  0.35483115 -0.56854487  0.54947555
  -0.64159743  0.627041    0.57270488 -0.41930282]
 [-0.26794088  0.42642914 -0.17784597  0.3948101  -0.30519136  0.53482056
  -0.06910253 -0.2971411  -0.0079182   0.64559942]
 [ 0.03115053 -0.11466124  0.46434142  0.59884694 -0.19509774 -0.22237391
   0.53421659 -0.46438837 -0.42732108 -0.38033892]
 [-0.0809643  -0.42718999  0.27230059 -0.28478963  0.5400966   0.27655912
  -0.50921203  0.59228948 -0.31951912  0.55684001]]
128.80699809227733
[[ 0.41695478  0.41700873 -0.15819499  0.35427795 -0.573568    0.54840572
  -0.64478165  0.62708051  0.57229105 -0.43521294]
 [-0.26879428  0.40477274 -0.18313835  0.39060064 -0.29594381  0.5406369
  -0.06064422 -0.29990491 -0.00401338  0.63172763]
 [ 0.03218189 -0.09620552  0.46562149  0.5979791  -0.2036191  -0.22581351
   0.52658462 -0.46393674 -0.43071028 -0.39365584]
 [-0.08016523 -0.41137183  0.26928114 -0.28871637  0.53462664  0.27466043
  -0.51416675  0.58950526 -0.322

[[ 0.42415279  0.47876188 -0.16144829  0.36069557 -0.58685251  0.55156072
  -0.64841906  0.63579365  0.57997583 -0.51564426]
 [-0.27497422  0.31319255 -0.1756938   0.37126815 -0.26771951  0.53854543
  -0.05334769 -0.31854661 -0.00979477  0.53153218]
 [ 0.03871202 -0.04255544  0.47217289  0.58558005 -0.21123249 -0.22010542
   0.51754244 -0.46898469 -0.42848202 -0.43967595]
 [-0.07234918 -0.34334615  0.28176974 -0.32214847  0.53234053  0.28271
  -0.52207713  0.56271524 -0.32626565  0.42526937]]
0.09498455328747832
[[ 0.42427311  0.4818733  -0.16151051  0.36096307 -0.58711254  0.55159489
  -0.64851203  0.6361118   0.58015289 -0.51635145]
 [-0.27515561  0.30638578 -0.17505251  0.37031988 -0.26637358  0.53845197
  -0.05301521 -0.31968444 -0.01018898  0.52955607]
 [ 0.03895688 -0.04071767  0.47223687  0.58522175 -0.21157365 -0.2199511
   0.51719165 -0.46888494 -0.42826189 -0.43957793]
 [-0.0722092  -0.34185789  0.28218822 -0.32348289  0.53258889  0.28283062
  -0.52216963  0.56152141 -0.32653

[[ 0.42539651  0.51832716 -0.16241336  0.36423322 -0.58975098  0.55153784
  -0.64926553  0.63970968  0.58209094 -0.52379648]
 [-0.27649152  0.230835   -0.16616092  0.36007487 -0.2544168   0.53852833
  -0.05079926 -0.33043742 -0.01351501  0.50094037]
 [ 0.04164493 -0.01827221  0.47122788  0.58257932 -0.21527801 -0.21883476
   0.51402577 -0.46590763 -0.42526513 -0.43442893]
 [-0.0721565  -0.3395192   0.28314289 -0.33565281  0.53424234  0.28318855
  -0.52220152  0.55039216 -0.32886397  0.41416377]]
0.06255622646992046
[[ 0.42544415  0.52021076 -0.16247299  0.36440093 -0.58987738  0.55151639
  -0.64929198  0.63988556  0.58218629 -0.52419342]
 [-0.27655457  0.22729382 -0.16573394  0.35962463 -0.25407723  0.53848652
  -0.05084011 -0.33081682 -0.01365833  0.49890127]
 [ 0.0417646  -0.01734023  0.47114225  0.58254525 -0.21542144 -0.21880953
   0.51392421 -0.4656706  -0.42510126 -0.43387489]
 [-0.07213275 -0.33934081  0.28312574 -0.33595786  0.53425485  0.28320142
  -0.52216819  0.55012047 -0.3

[[ 0.42601398  0.54323247 -0.16322171  0.36644747 -0.59149717  0.5512314
  -0.64959592  0.64202007  0.58337775 -0.52969513]
 [-0.27775819  0.18507649 -0.16045308  0.3520621  -0.24888686  0.53774458
  -0.05055809 -0.33656361 -0.01511598  0.47361791]
 [ 0.04311026 -0.00433596  0.46949396  0.58180311 -0.21720494 -0.21860255
   0.51266817 -0.4623147  -0.42273103 -0.42785608]
 [-0.07177683 -0.33488769  0.28244276 -0.34095868  0.53389939  0.2833008
  -0.52162071  0.54621793 -0.32929202  0.40629309]]
0.04972492556282603
[[ 0.42603026  0.54438597 -0.16326845  0.36652294 -0.59157848  0.55120917
  -0.64960963  0.64210106  0.58342713 -0.52998828]
 [-0.27780043  0.1830082  -0.16013998  0.35162389 -0.24861522  0.53769366
  -0.05054254 -0.33683121 -0.01516304  0.47203137]
 [ 0.04314559 -0.00369157  0.46935419  0.58175145 -0.21726529 -0.21860524
   0.51263576 -0.46214561 -0.42260067 -0.42749569]
 [-0.07174125 -0.3343762   0.28237675 -0.34126285  0.53384096  0.28329051
  -0.52159092  0.54603502 -0.329

[[ 0.42639582  0.55994348 -0.16383105  0.36770527 -0.59280782  0.55103536
  -0.64983409  0.64327437  0.58424513 -0.53510025]
 [-0.27867692  0.15762924 -0.15685031  0.3459588  -0.24505427  0.5364658
  -0.04995812 -0.33992949 -0.01567532  0.45089554]
 [ 0.04382238  0.00393219  0.46769328  0.58111384 -0.21838241 -0.21856338
   0.51195536 -0.45953779 -0.42099041 -0.42299684]
 [-0.07141511 -0.32827824  0.28147281 -0.34412052  0.53301406  0.28306332
  -0.52126794  0.54426512 -0.32956676  0.40131887]]
0.043556789089810456
[[ 0.42641664  0.5607736  -0.16386489  0.36776244 -0.59287845  0.55102857
  -0.64984479  0.64332646  0.58429211 -0.535434  ]
 [-0.27872018  0.1562844  -0.15668951  0.34565113 -0.24489879  0.53636585
  -0.04993273 -0.34005835 -0.01569803  0.44944327]
 [ 0.04385197  0.00429576  0.46759227  0.58107674 -0.2184587  -0.21856636
   0.51191045 -0.45938717 -0.42091539 -0.4227357 ]
 [-0.07139039 -0.32788796  0.28143578 -0.34425761  0.53295129  0.28305448
  -0.52126555  0.54417877 -0.3

[[ 0.42671712  0.57219919 -0.16433321  0.36862336 -0.59396329  0.55092855
  -0.65002174  0.64410466  0.58497279 -0.54003225]
 [-0.27931108  0.13921468 -0.1546804   0.34114663 -0.2428595   0.53481529
  -0.0494547  -0.34214545 -0.01601017  0.43151766]
 [ 0.04425698  0.00956413  0.46597657  0.58044902 -0.21957848 -0.21860561
   0.51137502 -0.45718858 -0.41968776 -0.41885353]
 [-0.0711181  -0.32181565  0.28074293 -0.34686457  0.53235763  0.28286012
  -0.5208961   0.54256624 -0.32971512  0.39628598]]
0.038616733523786945
[[ 0.42673875  0.57294975 -0.16436285  0.36868211 -0.59404404  0.55092281
  -0.65004081  0.6441593   0.58501714 -0.54036904]
 [-0.27934816  0.13828031 -0.15457117  0.34089135 -0.24272381  0.53469701
  -0.04940298 -0.34224303 -0.01600769  0.4304471 ]
 [ 0.04430088  0.0098855   0.46588566  0.580405   -0.21967645 -0.21859094
   0.51132548 -0.45705234 -0.41959268 -0.4187404 ]
 [-0.07111368 -0.32131682  0.28067085 -0.34704712  0.53230341  0.28282345
  -0.52087863  0.54245552 -0.

[[ 0.42704528  0.58284052 -0.16474702  0.36942815 -0.59509064  0.55088748
  -0.65021445  0.64485605  0.58564676 -0.54490829]
 [-0.27957912  0.126975   -0.15272028  0.33722808 -0.24097607  0.53312353
  -0.04894441 -0.34385155 -0.0159043   0.41381134]
 [ 0.0446966   0.01472941  0.46459799  0.5798397  -0.22072842 -0.21845515
   0.51087803 -0.45517389 -0.41837529 -0.41568115]
 [-0.07073709 -0.31586891  0.28014688 -0.34913656  0.53170651  0.28276183
  -0.52054458  0.54116602 -0.32967325  0.39027293]]
0.03564831626438898
[[ 0.42706411  0.58347371 -0.16476892  0.36947426 -0.59516062  0.55088369
  -0.65023055  0.64490238  0.58568346 -0.54519394]
 [-0.27961964  0.12639934 -0.1526285   0.33695899 -0.2408032   0.5330171
  -0.04886547 -0.34399917 -0.01588932  0.41261785]
 [ 0.04471101  0.01503458  0.46450059  0.57979029 -0.22081833 -0.21845249
   0.51083897 -0.45506983 -0.41829989 -0.41558828]
 [-0.07073537 -0.31545415  0.28009262 -0.34931797  0.53167244  0.28274726
  -0.52051508  0.54105036 -0.32

In [24]:
af_data_dic, af_label, label_af, af_data_dic_test, af_label_test, label_af_test = load_data(
              af_points=af_points,
              af_labels=af_labels,
              label_afs=label_afs)

hp = hyper_parameters(lamda=lamda, learning_rate=learning_rate,
                      epochs=epochs, batch_size=batch_size,
                      beta_1=beta_1, beta_2=beta_2, layers=layers)

n_params, sets = train_dev_test_sets(df_loc=df_loc, s_train=s_train, alpha_range=alpha_range,
                                     Re_range=Re_range, inputs_list=inputs_list, 
                                     outputs_list=outputs_list)

tf.reset_default_graph()
network = create_network(sets, label_af, af_data_dic, hp['layers'], hp['lamda'])

directory = os.getcwd() + '/'
saver = tf.train.Saver()
#imported_meta = tf.train.import_meta_graph(directory + "cp/cp.ckpt.meta")

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    #imported_meta.restore(sess, tf.train.latest_checkpoint('./'))
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, directory + "cp/cp_test.ckpt")
    print(sess.run(network['W' + str(len(network['layers']) - 1)]))
    test_test_set(network, sets, sess, af_data_dic_test, label_af_test, results='res.txt', label='test')

[[ 0.42711797  0.58473413 -0.16480501  0.36957988 -0.59529873  0.5508917
  -0.65024887  0.64501014  0.58577812 -0.54581209]
 [-0.27962258  0.12534697 -0.15236612  0.33652912 -0.24056951  0.53280423
  -0.04882352 -0.34418727 -0.01585986  0.41054176]
 [ 0.04475722  0.01571684  0.46433358  0.57970629 -0.22094074 -0.21842372
   0.51080355 -0.45482542 -0.41812456 -0.41518798]
 [-0.07068024 -0.31466079  0.28001976 -0.34960206  0.53156832  0.28273119
  -0.52048282  0.54089188 -0.32965521  0.38909881]]
